All code and files from my work on this project can be found publicly available on my [GitHub](https://github.com/Linueks/QuantumComputing/tree/main/IBM-quantum-challenge)

In [1]:
import numpy as np
import qiskit as qk
import qiskit.opflow as opflow
import matplotlib.pyplot as plt
from qiskit.quantum_info import state_fidelity
import qiskit.ignis.verification.tomography as tomo
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-whitegrid')

First we import the public libraries we will be using.

In [2]:
from decompositions import trotter_step_zyxzyx, trotter_step_zzyyxx, trotter_step_xplusy_zz_xplusy, trotter_step_xplusy_z_xplusy_z, trotter_step_xplusyplusz_xplusyplusz

decompositions = {
    'zyxzyx': trotter_step_zyxzyx,
    'zzyyxx': trotter_step_zzyyxx,
    'x+yzzx+y': trotter_step_xplusy_zz_xplusy,
    'x+yzx+yz': trotter_step_xplusy_z_xplusy_z,
    'x+y+z': trotter_step_xplusyplusz_xplusyplusz,
}

These are all the different Trotterizations that were investigated in this work. They are described in detail in the project [PDF](https://github.com/Linueks/QuantumComputing/blob/main/IBM-quantum-challenge/pub/IBM_Challenge_Spring_2022_Linus_Ekstrom.pdf) write up. Simply put, they are different reshufflings of the order the Pauli operators and the optimally constructed 2-qubit operators from the [Vatan, Williams paper](https://arxiv.org/abs/quant-ph/0308006). (decompositions needs to be in the same folder as this jupyter-notebook)

In [3]:
from simulation import TrotterSimulation

Next we are importing our custom Trotter simulation class. This allows us to set up lists of qiskit tomography circuits based on which decomposition we are interested in testing. It allows us full control over a fairly comprehensive list of variables. The workings of the class are also described in detail in the pdf write up, however seeing it in action is the easiest way to understand it. 

In [4]:
provider = qk.IBMQ.load_account()
provider = qk.IBMQ.get_provider(
    hub='ibm-q-community',
    group='ibmquantumawards',
    project='open-science-22'
)
# actual quantum computer backend
jakarta_backend = provider.get_backend('ibmq_jakarta')
# qiskit simulator backends
sim_jakarta_noiseless = qk.providers.aer.QasmSimulator()
sim_jakarta_noisy = qk.providers.aer.QasmSimulator.from_backend(
    jakarta_backend
)

Let us now see how to use our simulator class. We will create three instances: one with a real device as the backend, one with a noiseless simulation and one with a noisy simulator backend. To initialize our class we need to pass in the following parameters

    1. simulation_parameter
    2. simulation_backend
    3. backend_default_gates
    4. simulation_end_time
    5. number_of_qubits
    6. shots
    7. active_qubits
    8. verbose

The simulation parameter in our case will be time. The simulation backend is what we now will vary between our instances. Backend default gates is important to consider if we wanted to run our algorithm on a different quantum computer with different base gates. Simulation end time is when we want to simulate up to (starting from zero). Number of qubits speaks for itself. Shots is how many times we repeat our circuit to be able to pull out a probability distribution of our end states. Active qubits is a list of indeces of the qubits to be used for the algorithm, this is because in the competition it was specified we were to use the 1,3,5 index qubits on the Jakarta device. This functionality is something I had not kept in mind previously when doing quantum computing, but having the topology of the quantum computer in mind when doing algorithmic work seems to be of utmost importance! Verbose is just a boolean flag to enable various printouts when running the code.

Let's get to it! First we set up some variables


In [5]:
time = qk.circuit.Parameter('t')
shots = 8192
n_qubits = 7
end_time = np.pi                                                  # Specified in competition
basis_gates=['id', 'rz', 'sx', 'x', 'cx', 'reset']
active_qubits = [1, 3, 5]
verbose = True

In [6]:
min_trotter_steps = 4                                             # 4 minimum for competition
max_trotter_steps = 4
trotter_steps = range(min_trotter_steps, max_trotter_steps+1)     # Variable if just running one simulation

This might look a bit strange, but doing it this way let's us use one code to set up for both actual quantum computing and simulation. The reason is that when running on the real device there will always be a queue so if we want to loop over trotter steps we would only send the first job, and then we'd have to wait for that to complete until sending the next one. (might be a way around this, but this is how my code is written) When simulating the quantum device we don't have this big delay (at least for small system sizes)

In [7]:
simulator_jakarta_device_backend = TrotterSimulation(
    simulation_parameter=time,
    simulation_backend=jakarta_backend,
    backend_default_gates=basis_gates,
    simulation_end_time=end_time,
    number_of_qubits=n_qubits,
    shots=shots,
    active_qubits=active_qubits,
    verbose=verbose,
)

In [8]:
min_trotter_steps = 4                                             # 4 minimum for competition
max_trotter_steps = 12
trotter_steps = range(min_trotter_steps, max_trotter_steps+1)     # Variable if just running one simulation

In [9]:
simulator_noiseless_backend = TrotterSimulation(
    simulation_parameter=time,
    simulation_backend=sim_jakarta_noiseless,
    backend_default_gates=basis_gates,
    simulation_end_time=end_time,
    number_of_qubits=n_qubits,
    shots=shots,
    active_qubits=active_qubits,
    verbose=verbose,
)

In [10]:
simulator_noisy_backend = TrotterSimulation(
    simulation_parameter=time,
    simulation_backend=sim_jakarta_noisy,
    backend_default_gates=basis_gates,
    simulation_end_time=end_time,
    number_of_qubits=n_qubits,
    shots=shots,
    active_qubits=active_qubits,
    verbose=verbose,
)

We now have three instances of our class with different backends. The simulator_jakarta_device_backend instance will send a job with our chosen decomposition repeated for four Trotter steps, while the two others we can can loop over decompositions and trotter steps if we want to. Now let's see how to actually run the simulation using our instances.

The next step in the call sequence of our class is to set our desired qiskit transpilation level and whether we wish to enable symmetry protection or not. The theory of symmetry protection is covered in more detail in the [PDF](https://github.com/Linueks/QuantumComputing/blob/main/IBM-quantum-challenge/pub/IBM_Challenge_Spring_2022_Linus_Ekstrom.pdf). Basically, we rotate our error terms such that we potentially are able to cancel the leading order terms in the expansion. 

In [11]:
transpilation_level = 3
symmetry_protection = True
from decompositions import symmetry_protection_su_2

In [12]:
simulator_noisy_backend.set_transpilation_level(transpilation_level)
simulator_noisy_backend.set_symmetry_protection(
    symmetry_protection,
    symmetry_protection_su_2,
)

Transpilation level: 3
Symmetry Protection True


Change the simulator instance to use the other ones. Next in our call sequence all we have to do is run our simulation using the .run method. This will set up the base circuit and tomography circuits for a given decomposition. Let's see how it works!

In [13]:
trotter_steps = 8
fidelity = simulator_noisy_backend.run(
    name='zzyyxx',
    decomposition=trotter_step_zzyyxx,
    trotter_steps=trotter_steps,
)

Building base circuit with <function trotter_step_zzyyxx at 0x00000176BFB837B8> and 8 Trotter steps
Job ID: c0bfe56d-a8ab-478f-9604-abf1f165869f
Job Status: job has successfully run
state tomography fidelity = 0.1217
-----------------------------------------------------------


This is just an example of how running a circuit using my class works. It is flexible to be able to change the Trotter step freely by defining new functions. I have ran five jobs on the quantum device and the job ids are the following.

In [14]:
job_ids = [
    '624d6e13cfe45c3bf1e599f7',
    '624d74b7a5d4ee1e2677c35c',
    '624d75bed72033698a67ce88',
    '624d7642aacb9b785e5f4423',
    '624d76bdcaa2651ecaf19029',
]

Here we have the job ids from running the best performing decomposition and trotter step number combinations. Now we need to retrieve the results from IBMQ.

These jobs correspond to the following runs: 

    1. zzyyxx + symmetry protection with 4 trotter steps
    2. zzyyxx - symmetry protection with 4 trotter steps
    3. zyxzyx + symmetry protection with 7 trotter steps
    4. x+yzzx+y + symmetry protection with 8 trotter steps
    5. x+y+z - symmetry protection with 8 trotter steps


This next step of analysis will be painfully manually done. We will need to make an instance for each of the specifics of the runs and then generate the tomography circuits such that we can perform the full state tomography as stated in the competition. These were chosen because they were the ones that either performed the best in my simulated runs, or because they gave a big boost to the fidelity from symmetry protection. Again, more details can be found in the [PDF](https://github.com/Linueks/QuantumComputing/blob/main/IBM-quantum-challenge/pub/IBM_Challenge_Spring_2022_Linus_Ekstrom.pdf)

In [20]:
trotter_step_functions = {
    'zzyyxx1': trotter_step_zzyyxx,
    'zzyyxx2': trotter_step_zzyyxx,
    'zyxzyx': trotter_step_zyxzyx,
    'x+yzzx+y': trotter_step_xplusy_zz_xplusy,
    'x+y+z': trotter_step_xplusyplusz_xplusyplusz,
}

symmetry_protections = [
    True,
    False,
    True,
    True,
    False,
]

trotter_steps = [
    4,
    4,
    7,
    8,
    8,
]

In [22]:
simulator = TrotterSimulation(
    simulation_parameter=time,
    simulation_backend=jakarta_backend,
    backend_default_gates=basis_gates,
    simulation_end_time=end_time,
    number_of_qubits=n_qubits,
    shots=shots,
    active_qubits=active_qubits,
    verbose=False,
)
transpilation_level=3
simulator.set_transpilation_level(transpilation_level)


for i, (name, decomposition) in enumerate(trotter_step_functions.items()):
    simulator.set_symmetry_protection(
        symmetry_protection=symmetry_protections[i],
        symmetry_protection_function=symmetry_protection_su_2,
    )
    base_circuit = simulator.make_base_circuit(
        trotter_steps=trotter_steps[i],
        trotter_step_function=decomposition,
        name=name,
    )
    tomography_circuits = simulator.make_tomography_circuits(
        base_circuit,
    )
    job = jakarta_backend.retrieve_job(job_ids[i]) 
    fidelity = simulator.calculate_fidelity(
        job,
        tomography_circuits,
    )
    
    
    print(f'Fidelity: {np.round(fidelity, 6)} for {repr(decomposition)} with {trotter_steps[i]} trotter steps and symmetry protection {symmetry_protections[i]}')


Fidelity: 0.431886 for <function trotter_step_zzyyxx at 0x00000176BFB837B8> with 4 trotter steps and symmetry protection True
Fidelity: 0.064216 for <function trotter_step_zzyyxx at 0x00000176BFB837B8> with 4 trotter steps and symmetry protection False
Fidelity: 0.245432 for <function trotter_step_zyxzyx at 0x00000176BFB83730> with 7 trotter steps and symmetry protection True
Fidelity: 0.256461 for <function trotter_step_xplusy_zz_xplusy at 0x00000176BFB83510> with 8 trotter steps and symmetry protection True
Fidelity: 0.434357 for <function trotter_step_xplusyplusz_xplusyplusz at 0x00000176BFB83620> with 8 trotter steps and symmetry protection False


We see that our 'x+y+z' decomposition works the best, this makes sense as our Hamiltonian conserves SU2 and with how that Trotter step is constructed we also favour this. Next best is 'zzyyxx' with symmetry protection. 

That is basically it for code description. I do invite whoever reads this to check out my Github as well. If anyone wants to fork and use the code that would be great as well! I hope everyone reading this would be interested in checking out my PDF write up as well. It explains the theory and background needed to understand what's going on in the code. I have tried to write it fairly simply as doing so challenges me to understand the topics better! 

I have learned a lot during this work which I will surely carry forward thoroughout my further career in physics! It has been great fun to compete and actually complete this project! During this time I have read lots of papers in the quantum computing field. I would like to thank Alessandro Roggero, Morten Hjorth-Jensen and Stian Bilek for their great assistance and guidance!